<a href="https://colab.research.google.com/github/RobinHuumonen/Bird-classifier-with-Keras/blob/main/Bird_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
%matplotlib inline
import matplotlib.pyplot as plotter
import os

In [2]:
pip install -q pyyaml h5py 

In [3]:
rescaled_ImageDataGenerator = ImageDataGenerator(rescale=1./255) #rescale=1./255 scales RGB coefficients from 0–255 to 0–1

In [4]:
training_dir = '/content/drive/MyDrive/Colab Notebooks/250-bird-species-data/train'

In [5]:
validation_dir = '/content/drive/MyDrive/Colab Notebooks/250-bird-species-data/valid'

In [6]:
#NB: flow_from directory's default batch_size is 32
train_generator = rescaled_ImageDataGenerator.flow_from_directory(training_dir, target_size=(224,224))
validation_generator = rescaled_ImageDataGenerator.flow_from_directory(validation_dir, target_size=(224,224))

Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.


In [7]:
classes_amount = 250
pixel_height = pixel_width = 224
color_channels = 3

# VGG16 architechture implementation (src: https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c)
model = Sequential()
model.add(Conv2D(input_shape=(pixel_height,pixel_width,color_channels),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=classes_amount, activation="softmax"))

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [8]:
#Save model at some interval during the trainig process
filepath="/content/drive/MyDrive/Colab Notebooks/saved_models/Bird-classifier-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#Use early stopping callback to avoid over fitting
auto_stop_training = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights = True)

#CSVLogger logs epoch, acc, loss, val_acc, val_loss
logs = CSVLogger('logs.csv', separator=',', append=False)

In [ ]:
callbacks = [checkpoint, auto_stop_training, logs]
batch_size = 32
training_samples = 35215
validation_samples = 1250
model_name = 'Bird-classifier-model.h5'

history = model.fit(
        train_generator,
        steps_per_epoch = training_samples // batch_size,
        validation_data = validation_generator,
        validation_steps = validation_samples // batch_size,
        epochs = 100,
        verbose=1,
        callbacks=callbacks
)

model.save(model_name)

Epoch 1/100
 652/1100 [================>.............] - ETA: 3:08 - loss: 5.5434 - accuracy: 0.0066

In [ ]:
#Evaluating the model
train_loss, train_acc = model.evaluate_generator(train_generator, steps=16)
validation_loss, test_acc = model.evaluate_generator(validation_generator, steps=16)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
#Plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plotter.plot(epochs, loss, 'y', label='Training loss')
plotter.plot(epochs, val_loss, 'r', label='Validation loss')
plotter.title('Training and validation loss')
plotter.xlabel('Epochs')
plotter.ylabel('Loss')
plotter.legend()
plotter.show()


acc = history.history['acc']
val_acc = history.history['val_acc']
plotter.plot(epochs, acc, 'y', label='Training acc')
plotter.plot(epochs, val_acc, 'r', label='Validation acc')
plotter.title('Training and validation accuracy')
plotter.xlabel('Epochs')
plotter.ylabel('Accuracy')
plotter.legend()
plotter.show()

In [ ]:
#Continue training previously saved model
# =============================================================================
# from keras.models import load_model
# new_model = load_model(model_name)
# results = new_model.evaluate_generator(validation_generator, steps=16)
# print(" validation loss and accuracy are", results)
# new_model.fit_generator(
#         train_generator,
#         steps_per_epoch=training_samples // batch_size,
#         epochs=5,
#         validation_data=validation_generator,
#         validation_steps=validation_samples // batch_size,
#         callbacks=callbacks
# )
# model.save('Bird-classifier-model-updated.h5')
# =============================================================================

